# Module

In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../')
import os
import torch
import ollama
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)
from src.config import PATH_PROJECT, PROMPT_SYSTEM, PROMPT_MODEL_EXPERT

cpu


# Test models directly

In [9]:
from src.inference import get_answer
get_answer(patient_id=10, model='mistral')

' Based on the provided information, the diagnosis for this patient is Recurrent Depressive Disorder (RDD). The reason being that there is no history of manic, mixed or hypomanic episodes, and the observed symptoms align with depressive episodes (depressed_mood and reduced_energy), making a Bipolar disorder less likely. However, it is important to note that increased self-esteem can be associated with hypomania in some cases, but without a history of manic or hypomanic episodes, it is unlikely to be indicative of a bipolar disorder. A thorough evaluation and assessment by a mental health professional are necessary for an accurate diagnosis.'

In [ ]:
for i in range(1, 31):
    print(f"patient {i}", get_answer(patient_id=i, model='deepseek-r1:8b'))

In [ ]:
for i in range(1, 31):
    print(f"patient {i}", get_answer(patient_id=i, model='mistral'))

In [ ]:
for i in range(1, 31):
    print(f"patient {i}", get_answer(patient_id=i, model='llama3.1'))

# Test models with code

In [4]:
import pandas as pd 
df = pd.read_excel(os.path.join(PATH_PROJECT, "data/") + "dataset.xlsx").fillna("Non renseigné")

In [5]:
df['Observed_Symptom'].unique(), df['History_Condition'].unique()

(array(['depressed_mood', 'reduced_concentration', 'reduced_energy',
        'increased_talkativeness', 'diminished_interest_pleasure',
        'low_self_worth', 'psychomotor_disturbances',
        'increased_activity_energy', 'euphoria_irritability_expansiveness',
        'racing_thoughts', 'increased_self_esteem',
        'disrupted_excessive_sleep', 'change_in_appetite_weight',
        'decreased_need_for_sleep', 'distractibility',
        'recurrent_thoughts_death_suicide', 'impulsive_reckless_behavior',
        'increased_sexual_sociability_goal_directed_activity',
        'hopelessness', 'delusions', 'passivity_experiences',
        'disorganized_behavior'], dtype=object),
 array(['depressive', 'hypomanic', 'Non renseigné', 'mixed', 'manic'],
       dtype=object))

In [6]:
import requests

def test_ollama():
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("Ollama server is UP!")
        else:
            print(f"Ollama server responded with status code: {response.status_code}")
    except requests.exceptions.ConnectionError:
        print("Ollama server is DOWN or unreachable on localhost:11434")

test_ollama()

Ollama server is UP!


In [7]:
from src.inference import get_code, get_code_with_rag

models = ["mistral", "deepseek-coder-v2", "deepseek-r1", "llama3.2"]
models_expert = [ "mistral","deepseek-coder-v2", "deepseek-r1", "llama3.2" ]


c:\Users\victo\Documents\LLM\mva-llm-project\notebooks\..\src\inference.py:284: SyntaxWarning: invalid escape sequence '\c'
  user = f'''Now, translate the following criteria into python code between <code> and <\code> and add your explanations for Bipolar I, Bipolar II, Single Episode Depressive Disorder, and Recurrent Depressive Disorder. IT MUST BE IN PYTHON WITH THE PACKAGE pyDatalog. {retrieved_context} Relevant symptom names for Observed relation: {df['Observed_Symptom'].unique()}  Relevant condition names for History relation: {df['History_Condition'].unique()}'''


In [8]:
get_code("mistral",model_expert="deepseek-coder-v2", w_model_expert=True, prompt_system=PROMPT_SYSTEM, prompt_system_expert=PROMPT_MODEL_EXPERT,lan_logic='python')

 Here is a Python Pandas code to determine the diagnosis based on the provided criteria:

```python
import pandas as pd

# Assuming df is your DataFrame and it has the structure you've mentioned
def diagnose(df):
    # Define functions for each type of mood episode
    def depressive_episode(row):
        return (row['Mood_Episode'] == 'Depressive Episode') & \
               (row['Observed_Symptoms_Weeks'][0][0] == 'depressed_mood' and row['Observed_Symptoms_Weeks'][0][1] >= 2) & \
               row['History_Condition'].str.contains('|'.join(['Non renseigne', 'manic'])) & \
               (row['Observed_Symptoms_Weeks'][1:] != [(nan, nan)]).all()

    def manic_episode(row):
        return (row['Mood_Episode'] == 'Manic Episode') & \
               row['Observed_Symptoms_Weeks'][0][0] in ['euphoria_irritability_expansiveness', 'increased_talkativeness'] & \
               len(row[row['Observed_Symptoms_Weeks'][1:]]) >= 3 & \
               (row['History_Condition'].str.contains('|'.j

In [9]:
df.head()

PatientID Disorder         Observed_Symptom  Observed_Week  \
0          1     BPD2           depressed_mood            1.5   
1          1     BPD2    reduced_concentration            1.2   
2          1     BPD2           reduced_energy            0.8   
3          1     BPD2  increased_talkativeness            0.6   
4          2      RDD           depressed_mood            5.7   

  History_Condition  History_Count   Mood_Episode  
0        depressive            1.0  Non renseigné  
1         hypomanic            1.0  Non renseigné  
2     Non renseigné  Non renseigné  Non renseigné  
3     Non renseigné  Non renseigné  Non renseigné  
4        depressive            1.0     depressive

In [10]:
def has_criteria(series, criteria):
    for sym in criteria:
        if not series[sym].any():
            return False
    return True

def is_depressive_episode(series):
    return (series['depressed_mood'].any() and
            has_criteria(series, ['reduced_concentration', 'feelings_of_worthlessness_or_excessive_guilt',
                                'hopelessness', 'thoughts_of_death_or_suicidal_ideation',
                                'insomnia_or_hypersomnia',
                                'significant_weight_change_or_appetite_disturbance',
                                'psychomotor_agitation_or_retardation',
                                'fatigue_or_loss_of_energy']))

def is_manic_episode(series):
    return (series['euphoria_irritability_expansiveness'].any() and
            series['increased_activity_energy'].any() and
            has_criteria(series, ['inflated_self_esteem_or_grandiosity',
                                'decreased_need_for_sleep',
                                'more_talkative_or_pressured_speech',
                                'flight_of_ideas_or_racing_thoughts',
                                'distractibility',
                                'increased_goal-directed_activity_or_psychomotor_agitation',
                                'risky_behaviors']))

def is_hypomanic_episode(series):
    return (series['euphoria_irritability_expansiveness'].any() and
            series['increased_activity_energy'].any() and
            has_criteria(series, ['inflated_self_esteem', 'decreased_need_for_sleep',
                                'more_talkative_or_pressured_speech',
                                'flight_of_ideas', 'distractibility',
                                'increased_goal-directed_activity']))

def is_mixed_episode(series):
    return (series['depressed_mood'].any() and series['euphoria_irritability_expansiveness'].any())

# Define helper functions for diagnosis
def get_episodes_duration(series):
    episodes = []
    for episode in df.loc[df['symptom'] == series['condition'], 'duration'].unique():
        episodes += [episode] * len(df[(df['symptom'] == series['condition']) & (df['duration'] == episode)])
    return pd.Series(episodes, name='duration')

def get_history(series):
    conditions = []
    for condition in df[df['symptom'].isin(series['condition'])]['condition'].unique():
        conditions += [condition] * len(df[(df['condition'] == condition) & (df['symptom'] == series['condition'])])
    return pd.Series(conditions, name='condition')

# Diagnose the patient based on their symptoms and history of mood episodes
def diagnose(series):
    if is_depressive_episode(series) and df.loc[df['symptom'] == 'Non renseigné', 'duration'].iloc[0] not in series['duration']:
        return 'Single Episode Depressive Disorder'
    elif is_depressive_episode(series) and len(df.loc[(df['symptom'] == 'Non renseigné') & (df['duration'] == df.loc[df['condition'] == 'depressive', 'duration'].iloc[0])]['condition']) > 1:
        return 'Recurrent Depressive Disorder'
    elif is_manic_episode(series):
        if series['condition'].iloc[0] == 'Non renseigné':
            conditions = get_history(series)
            if len(conditions) > 1 and 'hypomanic' in conditions:
                return 'Bipolar I Disorder'
            elif all([c != 'manic' for c in conditions]):
                return 'Unspecified Mood Disorder with Manic Episode'
        else:
            if len(df[(df['symptom'] == series['condition']) & (df['duration'] == df.loc[df['condition'] == 'manic', 'duration'].iloc[0])]) > 1:
                return 'Bipolar I Disorder'
    elif is_hypomanic_episode(series):
        if series['condition'].iloc[0] == 'Non renseigné':
            conditions = get_history(series)
            if len(conditions) > 1 and 'hypomanic' in conditions:
                return 'Bipolar II Disorder'
            elif all([c != 'hypomanic' for c in conditions]):
                return 'Unspecified Mood Disorder with Hypomanic Episode'
        else:
            if len(df[(df['symptom'] == series['condition']) & (df['duration'] == df.loc[df['condition'] == 'hypomanic', 'duration'].iloc[0])]) > 1:
                return 'Bipolar II Disorder'
    elif is_mixed_episode(series):
        if series['condition'].iloc[0] == 'Non renseigné':
            conditions = get_history(series)
            if len(conditions) > 1 and ('hypomanic' in conditions or 'manic' in conditions):
                return 'Bipolar I Disorder with Mixed Features'
        else:
            if len(df[(df['symptom'] == series['condition']) & (df['duration'] == df.loc[df['condition'] == 'mixed', 'duration'].iloc[0])]) > 1:
                return 'Bipolar I Disorder with Mixed Features'
    else:
        return 'Cannot determine diagnosis'

In [26]:
df.head()

PatientID Disorder         Observed_Symptom  Observed_Week  \
0          1     BPD2           depressed_mood            1.5   
1          1     BPD2    reduced_concentration            1.2   
2          1     BPD2           reduced_energy            0.8   
3          1     BPD2  increased_talkativeness            0.6   
4          2      RDD           depressed_mood            5.7   

  History_Condition  History_Count   Mood_Episode  
0        depressive            1.0  Non renseigné  
1         hypomanic            1.0  Non renseigné  
2     Non renseigné  Non renseigné  Non renseigné  
3     Non renseigné  Non renseigné  Non renseigné  
4        depressive            1.0     depressive

In [25]:
df['diagnosis'] = df.apply(diagnose, axis=1)

KeyError: 'depressed_mood'

In [ ]:
import numpy as np
def diagnose(df):
    df['Depressive Episode'] = df['Observed_Symptom'].apply(lambda x: 1 if all([sym in x for sym in ['depressed_mood', 'diminished_interest_pleasure']]) else 0)
    df['Hypomanic Episode'] = df['History_Count'] > 0
    df['Manic Episode'] = df['Hypomanic Episode'] & (df['Observed_Symptom'].apply(lambda x: len([sym in x for sym in ['euphoria_irritability_expansiveness', 'increased_activity_energy', 'impulsive_reckless_behavior']]) >= 3) == True)
    df['Mixed Episode'] = df['Depressive Episode'] & (df['Manic Episode'] | df['Hypomanic Episode'])

    df['Bipolar I Disorder'] = df['Manic Episode'].astype(int) + df['Mixed Episode'].astype(int) > 0
    df['Bipolar II Disorder'] = (df['Hypomanic Episode'].astype(int) == 1) & (df['Depressive Episode'].astype(int) >= 1)
    df['Single Episode Depressive Disorder'] = (df['Depressive Episode'].astype(int) == 1) & (df['Hypomanic Episode'].astype(int) == 0)
    df['Recurrent Depressive Disorder'] = (df['Depressive Episode'].cumsum() > 1) & (df['Hypomanic Episode'].eq(np.nan).all())


In [11]:
from src import dataprep
df_grouped = dataprep.preproc_df()
df_grouped[df_grouped['PatientID'] == 1]

PatientID Disorder                                  History_Condition  \
0          1     BPD2  [depressive, hypomanic, Non renseigne, Non ren...   

          History_Count                                       Mood_Episode  \
0  [1.0, 1.0, nan, nan]  [Non renseigne, Non renseigne, Non renseigne, ...   

                             Observed_Symptoms_Weeks  
0  [(depressed_mood, 1.5), (reduced_concentration...

In [ ]:
diagnose(df)

PatientID       Disorder                     Observed_Symptom  \
0            1           BPD2                       depressed_mood   
1            1           BPD2                reduced_concentration   
2            1           BPD2                       reduced_energy   
3            1           BPD2              increased_talkativeness   
4            2            RDD                       depressed_mood   
..         ...            ...                                  ...   
134         30  Non renseigné  euphoria_irritability_expansiveness   
135         30  Non renseigné            increased_activity_energy   
136         30  Non renseigné              increased_talkativeness   
137         30  Non renseigné                      racing_thoughts   
138         30  Non renseigné             decreased_need_for_sleep   

     Observed_Week History_Condition  History_Count   Mood_Episode  \
0              1.5        depressive            1.0  Non renseigné   
1              1.2         hypomanic            1.0  Non renseigné   
2              0.8     Non renseigné            NaN  Non renseigné   
3              0.6     Non renseigné            NaN  Non renseigné   
4              5.7        depressive            1.0     depressive   
..             ...               ...            ...            ...   
134            0.7     Non renseigné            NaN      hypomanic   
135            1.2     Non renseigné            NaN  Non renseigné   
136            1.8     Non renseigné            NaN  Non renseigné   
137            0.7     Non renseigné            NaN  Non renseigné   
138            1.2     Non renseigné            NaN  Non renseigné   

     Depressive Episode  Hypomanic Episode  Manic Episode  Mixed Episode  \
0                     0               True           True          False   
1                     0               True           True          False   
2                     0              False          False          False   
3                     0              False          False          False   
4                     0               True           True          False   
..                  ...                ...            ...            ...   
134                   0              False          False          False   
135                   0              False          False          False   
136                   0              False          False          False   
137                   0              False          False          False   
138                   0              False          False          False   

     Bipolar I Disorder  Bipolar II Disorder  \
0                  True                False   
1                  True                False   
2                 False                False   
3                 False                False   
4                  True                False   
..                  ...                  ...   
134               False                False   
135               False                False   
136               False                False   
137               False                False   
138               False                False   

     Single Episode Depressive Disorder  Recurrent Depressive Disorder  
0                                 False                          False  
1                                 False                          False  
2                                 False                          False  
3                                 False                          False  
4                                 False                          False  
..                                  ...                            ...  
134                               False                          False  
135                               False                          False  
136                               False                          False  
137                               False                          False  
138                           

In [12]:
def diagnose_mood_disorders(df):
    df['Depressive Episode'] = (
        (df['depressed_mood'] == 1) &
        (df['reduced_concentration'] == 1 | df['reduced_energy'] == 1) &
        (df['feelings_of_worthlessness_or_excessive_guilt'] == 1 |
         df['hopelessness'] == 1 | df['thoughts_of_death_or_suicidal_ideation'] == 1) &
        (df['Insomnia_or_hypersomnia'] == 1 | df['significant_weight_change_or_appetite_disturbance'] == 1 |
         df['psychomotor_agitation_or_retardation'] == 1 | df['Fatigue_or_loss_of_energy'] == 1) &
        (df['Mood_Episode'] == 'Depressive Episode') &
        (df['History_Condition'].str.contains('depressive', case=False, na=False) |
         df['History_Count'] > 0) &
        (df['History_Condition'].str.contains('Non renseigné', case=False, na=True) == False)
    )

    df['Manic Episode'] = (
        (df['euphoria_irritability_expansiveness'] == 1 | df['increased_talkativeness'] == 1) &
        (df['Increased_activity_energy'] == 1 | df['Decreased_need_for_sleep'] == 1) &
        (df['racing_thoughts'] == 1 | df['distractibility'] == 1) &
        (df['Mood_Episode'] == 'Manic Episode') &
        (df['History_Condition'].str.contains('manic', case=False, na=False) |
         df['History_Count'] > 0) &
        (df['History_Condition'].str.contains('Non renseigné', case=False, na=True) == False)
    )

    df['Mixed Episode'] = (
        (df['Manic Episode'] == True) & (df['Depressive Episode'] == True) &
        (df['Mood_Episode'] == 'Mixed Episode') &
        (df['History_Condition'].str.contains('mixed', case=False, na=False) |
         df['History_Count'] > 0) &
        (df['History_Condition'].str.contains('Non renseigné', case=False, na=True) == False)
    )

    def bipolar_I(x):
        return x['Manic Episode'].any() and (x['Mixed Episode'].sum() > 0 or x['Depressive Episode'].sum() > 0)

    df['Bipolar I Disorder'] = df.apply(bipolar_I, axis=1)

    def bipolar_II(x):
        return (x['hypomanic'] == True) & x['depressive'].sum() > 0 and not x['Manic Episode'].any()

    df['Bipolar II Disorder'] = df.apply(bipolar_II, axis=1)

    def single_episode(x):
        return (x['Depressive Episode'] == True) & ~ (x['hypomanic'].any() | x['Manic Episode'].any())

    df['Single Episode Depressive Disorder'] = df.apply(single_episode, axis=1)

    def recurrent_depressive(x):
        return len(df[df['Depressive Episode'] == True]['PatientID']) > 1 and not (x['hypomanic'].any() | x['Manic Episode'].any())

    df['Recurrent Depressive Disorder'] = df.apply(recurrent_depressive, axis=1)

    return df


In [13]:
df.head()

PatientID Disorder         Observed_Symptom  Observed_Week  \
0          1     BPD2           depressed_mood            1.5   
1          1     BPD2    reduced_concentration            1.2   
2          1     BPD2           reduced_energy            0.8   
3          1     BPD2  increased_talkativeness            0.6   
4          2      RDD           depressed_mood            5.7   

  History_Condition  History_Count   Mood_Episode  
0        depressive            1.0  Non renseigné  
1         hypomanic            1.0  Non renseigné  
2     Non renseigné  Non renseigné  Non renseigné  
3     Non renseigné  Non renseigné  Non renseigné  
4        depressive            1.0     depressive

In [14]:
diagnose_mood_disorders(df)

KeyError: 'depressed_mood'

In [12]:
def diagnose(df):
    # Define functions for each type of mood episode
    def depressive_episode(row):
        return (row['Mood_Episode'] == 'Depressive Episode') & \
               (row['Observed_Symptoms_Weeks'][0][0] == 'depressed_mood' and row['Observed_Symptoms_Weeks'][0][1] >= 2) & \
               row['History_Condition'].str.contains('|'.join(['Non renseigne', 'manic'])) & \
               (row['Observed_Symptoms_Weeks'][1:] != [(nan, nan)]).all()

    def manic_episode(row):
        return (row['Mood_Episode'] == 'Manic Episode') & \
               row['Observed_Symptoms_Weeks'][0][0] in ['euphoria_irritability_expansiveness', 'increased_talkativeness'] & \
               len(row[row['Observed_Symptoms_Weeks'][1:]]) >= 3 & \
               (row['History_Condition'].str.contains('|'.join(['Non renseigne', 'manic'])) | row['Impairment'] > 0)

    def mixed_episode(row):
        return (row['Mood_Episode'] == 'Mixed Episode') & \
               row['Observed_Symptoms_Weeks'][0][0] in ['depressed_mood', 'euphoria_irritability_expansiveness'] & \
               row['History_Condition'].str.contains('|'.join(['Non renseigne']))

    def hypomanic_episode(row):
        return (row['Mood_Episode'] == 'Hypomanic Episode') & \
               row['Observed_Symptoms_Weeks'][0][0] in ['euphoria_irritability_expansiveness', 'increased_talkativeness'] & \
               len(row[row['Observed_Symptoms_Weeks'][1:]]) >= 3 & \
               row['History_Condition'].str.contains('|'.join(['Non renseigne'])) & \
               row['Impairment'] <= 0

    def bipolar_i(row):
        return (row['Disorder'] == 'Bipolar I Disorder') & \
               row['History_Condition'].str.contains('|'.join(['manic', 'mixed'])) & \
               row['History_Count'][1] >= 1

    def bipolar_ii(row):
        return (row['Disorder'] == 'Bipolar II Disorder') & \
               row['History_Condition'].str.contains('|'.join(['hypomanic', 'mixed'])) & \
               row['History_Count'][1] >= 2 & row['History_Count'][0] >= 1 & row['History_Count'][1] == 1

    def single_episode(row):
        return (row['Disorder'] == 'Single Episode Depressive Disorder') & \
               depressive_episode(row) & row['History_Count'] == [1.0, nan, nan, nan]

    def recurrent_depressive(row):
        return (row['Disorder'] != 'Single Episode Depressive Disorder') & \
               depressive_episode(row) & (row['History_Count'][0] > 1 | row['History_Count'][2] >= 2)

    # Add columns for mood episodes and disorders based on the defined functions
    df['Depressive Episode'] = df.apply(depressive_episode, axis=1)
    df['Manic Episode'] = df.apply(manic_episode, axis=1)
    df['Hypomanic Episode'] = df.apply(hypomanic_episode, axis=1)
    df['Mixed Episode'] = df.apply(mixed_episode, axis=1)
    df['Bipolar I Disorder'] = df.apply(bipolar_i, axis=1)
    df['Bipolar II Disorder'] = df.apply(bipolar_ii, axis=1)
    df['Single Episode Depressive Disorder'] = df.apply(single_episode, axis=1)
    df['Recurrent Depressive Disorder'] = df.apply(recurrent_depressive, axis=1)

In [13]:
diagnose(df_grouped)

AttributeError: 'list' object has no attribute 'str'